In [14]:
import ast
import pandas as pd
import os
import subprocess

archivo = '/Users/luisalbertocerelli/Desktop/00-Todo/Data Science/01-Full Time/TI_1/Datasets_ORIGINALES/json descomprimidos/3_users_items.json'

data = []

with open(archivo, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            json_data = ast.literal_eval(line)
            data.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

df_3_users_items = pd.DataFrame(data)

# Hacemos un csv para ver los datos originales:
df_3_users_items.to_csv('/Users/luisalbertocerelli/Desktop/00-Todo/Data Science/01-Full Time/TI_1/Datasets_ORIGINALES/csv/users_items_ORIGINAL.csv', index=False)


# Hacemos una copia para continuar trabajando con ella y no perder los datos originales:
df_3_users_items_copy = df_3_users_items.copy()

# Con el objeto de trabajar con datos de calidad, sacaremos las filas que contengan listas vacías en la columna 'items' del tipo[], pero para ello debemos convertir los valores de la columna 'items' a cadenas, ya que actualmente son listas.

# Convertir los valores de la columna 'items' a cadenas
df_3_users_items_copy['items'] = df_3_users_items_copy['items'].astype(str)

# Eliminar las filas donde la celda en la columna 'items' sea una lista vacía o una cadena que contiene solo corchetes
df_3_users_items_copy[df_3_users_items_copy['items'].str.strip("[]").str.strip() != ""]

# Ahora df contendrá solo las filas donde la celda en la columna 'items' no sea una lista vacía ni una cadena que contenga solo corchetes

#Convertir la columna 'items' a una lista de diccionarios:

df_3_users_items_copy['items'] = df_3_users_items_copy['items'].apply(ast.literal_eval)

#La función ast.literal_eval convierte el string en una estructura de datos de Python (en este caso, una lista de diccionarios).

# Crear un nuevo DataFrame con todas las filas explodidas
df_3_users_items_copy1 = df_3_users_items_copy.explode('items', ignore_index=True)

# Crear un DataFrame con los diccionarios normalizados
df_3_users_items_copy2 = pd.json_normalize(df_3_users_items_copy1['items'])

# Agregar las columnas adicionales del DataFrame original
df_3_users_items_copy3 = pd.concat([df_3_users_items_copy1, df_3_users_items_copy2], axis=1)

# Eliminar las columnas 'steam_id', 'user_url', 'items' y 'playtime_2weeks' ya que no las necesitamos para el análisis:

df_3_users_items_copy4 = df_3_users_items_copy3.drop(['steam_id', 'user_url', 'items', 'playtime_2weeks'], axis=1)

# Dejaremos solo 2000 filas elegidas al azar para hacer pruebas con menos datos y un random_state chico para no perder informacion dado el tipo de dataset que tenemos:
df_3_users_items_copy5=df_3_users_items_copy4.sample(2000, random_state=8)

# Y reindexamos los indices:
df_3_users_items_copy5.reset_index(drop=True, inplace=True)

 # Lo comprimimos a parquet, colocandolo en la carpeta de datasets para render y su nombre original:
df_3_users_items_copy5.to_parquet('/Users/luisalbertocerelli/Desktop/00-Todo/Data Science/01-Full Time/TI_1/Datasets_para_Render/users_items.parquet')

# Generamos un CSV con los datos:
df_3_users_items_copy5.to_csv('/Users/luisalbertocerelli/Desktop/00-Todo/Data Science/01-Full Time/TI_1/Datasets_para_Render/users_items.csv', index=False)

 # Imprimimos entonces el exito de la operacion y dejamos el link para acceder al archivo:
print('Archivo users_items.parquet y users_items.csv creado con exito!')

# Y dejamos el link para acceder a los archivos, para ello definimos las rutas:
file_path = os.path.abspath('/Users/luisalbertocerelli/Desktop/00-Todo/Data Science/01-Full Time/TI_1/Datasets_para_Render/users_items.parquet')

# Define la ruta del archivo CSV
csv_file_path = os.path.abspath('/Users/luisalbertocerelli/Desktop/00-Todo/Data Science/01-Full Time/TI_1/Datasets_para_Render/users_items.csv')

# Abre la ubicación del archivo CSV en Finder
subprocess.run(['open', '-R', csv_file_path])

# Abre la ubicación del archivo en Finder (Mac)
subprocess.run(['open', '-R', file_path])



Archivo users_items.parquet y users_items.csv creado con exito!


CompletedProcess(args=['open', '-R', '/Users/luisalbertocerelli/Desktop/00-Todo/Data Science/01-Full Time/TI_1/Datasets_para_Render/users_items.parquet'], returncode=0)

In [3]:
# Vemos el peso en kb de los archivos generados:
print(f'Peso del archivo users_items.parquet: {os.path.getsize(file_path) / 1024:.2f} KB')
print(f'Peso del archivo users_items.csv: {os.path.getsize(csv_file_path) / 1024:.2f} KB')


Peso del archivo users_items.parquet: 28664.71 KB
Peso del archivo users_items.csv: 248710.33 KB
